# Evaluate Aliases of Transformed EMA Registries

This notebook prepares and explores alias pairs for LLM assessment, focusing on EMA registries that have been assigned to clusters.

## 1. Setup & Imports

In [23]:
import os
import pandas as pd
import numpy as np
from pathlib import Path
from tqdm import tqdm

# Set working directory (adjust if needed)
working_dir = '/home/gpinon/more_europa/clean_rdc_experiments/projects/P05_refine_dedup'
os.chdir(working_dir)
print(f'Changed working directory to {working_dir}')

Changed working directory to /home/gpinon/more_europa/clean_rdc_experiments/projects/P05_refine_dedup


## 2. Load Data

In [24]:
# Load EMA prediction results (output of R03)
ema_pred_xlsx = 'data/W02/R03_evaluate_model_performance_on_ema_registries/ema_prediction_results.xlsx'
ema_pred_df = pd.read_excel(ema_pred_xlsx)
print(f'Loaded EMA prediction results: {ema_pred_df.shape}')

# Load clusters table
clusters_table_xlsx = 'data/W02/R02_evaluate_model_performance/clusters_table.xlsx'
clusters_df = pd.read_excel(clusters_table_xlsx)
print(f'Loaded clusters table: {clusters_df.shape}')

Loaded EMA prediction results: (237, 10)
Loaded clusters table: (54335, 3)


## 3. Prepare Alias Pairs for LLM Assessment
For each EMA registry that landed in a cluster, collect all aliases in the same cluster (≤50/all, >50/top 50 by frequency).

In [ ]:
# # Merge clusters info for alias lookup
# clusters_df['Final_Cluster'] = clusters_df['Final_Cluster'].astype(str)
# # Only consider EMA registries that are not noise
# def is_noise(cluster):
#     return str(cluster) == '0' or str(cluster).endswith('_0')
# ema_transformed_df = ema_pred_df[~ema_pred_df['assigned_cluster'].apply(is_noise)].copy()
# print(f'Number of transformed EMA registries: {ema_transformed_df.shape[0]}')
# # Prepare alias pairs
# alias_pairs = []
# max= 1  # Maximum number of aliases per EMA registry
# for idx, row in tqdm(ema_transformed_df.iterrows(), total=ema_transformed_df.shape[0]):
#     ema_name = row['ema_full_name']
#     ema_object_id = row.get('ema_object_id', None)
#     cluster_id = row['assigned_cluster']
#     # Get all aliases in the same cluster
#     aliases = clusters_df[clusters_df['Final_Cluster'] == cluster_id]
#     # Exclude the EMA registry itself if present
#     aliases = aliases[aliases['full_name'] != ema_name]
#     # Sort aliases by number_of_occurrences
#     aliases = aliases.sort_values('number_of_occurrences', ascending=False)
#     # If cluster size > 50, take top 50
#     if len(aliases) > max:
#         aliases = aliases.head(max)
#     for _, alias_row in aliases.iterrows():
#         alias_pairs.append({
#             'ema_full_name': ema_name,
#             'assigned_cluster': cluster_id,
#             'alias': alias_row['full_name'],
#             'alias_nb_occ': alias_row['number_of_occurrences'],
#             'ema_object_id': ema_object_id
#         })
# alias_pairs_df = pd.DataFrame(alias_pairs)
# print(f'Prepared {alias_pairs_df.shape[0]} EMA-alias pairs for LLM assessment.')

Number of transformed EMA registries: 73


100%|██████████| 73/73 [00:00<00:00, 120.37it/s]

Prepared 72 EMA-alias pairs for LLM assessment.


In [ ]:
# Merge clusters info for alias lookup
clusters_df['Final_Cluster'] = clusters_df['Final_Cluster'].astype(str)
# Only consider EMA registries that are not noise
def is_noise(cluster):
    return str(cluster) == '0' or str(cluster).endswith('_0')
ema_transformed_df = ema_pred_df[ema_pred_df['assigned_cluster'].apply(is_noise)].copy()
print(f'Number of transformed EMA registries: {ema_transformed_df.shape[0]}')
# Prepare alias pairs
alias_pairs = []
max= 1  # Maximum number of aliases per EMA registry
for idx, row in tqdm(ema_transformed_df.iterrows(), total=ema_transformed_df.shape[0]):
    ema_name = row['ema_full_name']
    ema_object_id = row.get('ema_object_id', None)
    cluster_id = row['assigned_cluster']
    # # Get all aliases in the same cluster
    aliases = ema_transformed_df[ema_transformed_df['ema_full_name'] == ema_name]
    # Exclude the EMA registry itself if present
    aliases = aliases[aliases['closest'] != ema_name]
    # # Sort aliases by number_of_occurrences
    # aliases = aliases.sort_values('number_of_occurrences', ascending=False)
    # If cluster size > 50, take top 50
    if len(aliases) > max:
        aliases = aliases.head(max)
    for _, alias_row in aliases.iterrows():
        alias_pairs.append({
            'ema_full_name': ema_name,
            'assigned_cluster': cluster_id,
            'alias': alias_row['closest'],
            'alias_nb_occ': alias_row['closest_nb_occ'],
            'ema_object_id': ema_object_id
        })
alias_pairs_df = pd.DataFrame(alias_pairs)
print(f'Prepared {alias_pairs_df.shape[0]} EMA-alias pairs for LLM assessment.')

Number of transformed EMA registries: 73


100%|██████████| 73/73 [00:00<00:00, 1303.75it/s]

Prepared 70 EMA-alias pairs for LLM assessment.


## 4. LLM Judgement: Prepare Prompts and Run Inference
Load the model config and prompt, prepare the prompts, make inferences on all prepared pairs, and parse the results.

In [37]:
# # slected first 5 for testing
# alias_pairs_df = alias_pairs_df.head(5)

## Prepare prompts

In [49]:
import json
from pathlib import Path
import os
from dotenv import load_dotenv

# Load prompt template
prompt_txt = 'etc/prompts/prompt_compare_registry_names.txt'
with open(prompt_txt, 'r') as pf:
    base_prompt = pf.read().strip()

# Load model config
model_config_path = 'etc/configs/gpt4_1_openai_config.json'  # adjust if needed
with open(model_config_path, 'r', encoding='utf-8') as f:
    model_cfg = json.load(f)

# Prepare prompts
def construct_prompt(base_prompt: str, name1: str, name2: str) -> str:
    return base_prompt.replace('{{content_a}}', name1).replace('{{content_b}}', name2)

prompts = []
for idx, row in alias_pairs_df.iterrows():
    prompt = construct_prompt(base_prompt, row['ema_full_name'], row['alias'])
    prompts.append({
        'prompt': prompt,
        'custom_id': f"{row['ema_full_name']}|||{row['alias']}",
        'ema_full_name': row['ema_full_name'],
        'alias': row['alias'],
        'assigned_cluster': row['assigned_cluster'],
        'alias_nb_occ': row['alias_nb_occ'],
        'ema_object_id': row.get('ema_object_id', None)
    })
print(f'Prepared {len(prompts)} prompts for LLM assessment.')

Prepared 70 prompts for LLM assessment.


## Make inferences

In [50]:
import logging
logging.basicConfig(
    level=logging.WARNING,
    format="%(asctime)s %(levelname)s %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S",
)
logger = logging.getLogger(__name__)
logging.getLogger("httpx").setLevel(logging.WARNING)

In [51]:
# LLM backend setup (same as in S03_eval_pairs_similarity_assessment_with_llm)
import llm_backends
from llm_inference.cache.tmp import TmpCacheStorage
import asyncio

load_dotenv()
cache_storage = TmpCacheStorage()
backend = llm_backends.OpenAIAsyncBackend(api_key=os.getenv('OPENAI_API_KEY'), cache_storage=cache_storage)

async def run_async_inference(prompts, backend, model_cfg):
    raw_responses = []
    pbar = tqdm(total=len(prompts), desc='LLM Inference')
    for prompt in prompts:
        raw_response = await backend.infer_one(prompt, model_cfg)
        raw_response['custom_id'] = prompt['custom_id']
        raw_responses.append(raw_response)
        pbar.update(1)
    pbar.close()
    return raw_responses

In [52]:
logging.warning("Starting asynchronous inference ...")
loop = asyncio.get_event_loop()
raw_responses = await run_async_inference(prompts, backend, model_cfg)
print(f'LLM inference complete. {len(raw_responses)} responses collected.')

2025-08-01 10:02:57,773 - root - WARNING - Starting asynchronous inference ...


LLM Inference: 100%|██████████| 70/70 [01:31<00:00,  1.30s/it]

LLM inference complete. 70 responses collected.


## Parse and collect results

In [53]:
# Parse and collect results
prompt_map = {p['custom_id']: p for p in prompts}
llm_responses = []
for raw_response in tqdm(raw_responses, desc='Processing responses', leave=False):
    custom_id = raw_response.get('custom_id', '')
    prompt_obj = prompt_map.get(custom_id)
    if prompt_obj:
        parsed_response = backend._parse_response(raw_response)
        parsed_response['custom_id'] = custom_id
        parsed_response['ema_full_name'] = prompt_obj['ema_full_name']
        parsed_response['alias'] = prompt_obj['alias']
        parsed_response['assigned_cluster'] = prompt_obj['assigned_cluster']
        parsed_response['alias_nb_occ'] = prompt_obj['alias_nb_occ']
        parsed_response['ema_object_id'] = prompt_obj['ema_object_id']
        llm_responses.append(parsed_response)
llm_responses_df = pd.DataFrame(llm_responses)
print(f'Parsed {llm_responses_df.shape[0]} LLM responses.')

Parsed 70 LLM responses.


# 5. Post processing

In [54]:
# Merge LLM responses into alias_pairs_df
merged_df = alias_pairs_df.merge(
    llm_responses_df,
    on=["ema_full_name", "alias", "assigned_cluster", "alias_nb_occ", "ema_object_id"],
    how="left",
)


# Assign final_label: 1 if LLM says 'same', 0 if 'different', else uncertain
def get_final_label(row):
    if "final_decision" in row and row["final_decision"] == "same":
        return 1
    elif "final_decision" in row and row["final_decision"] == "different":
        return 0
    return None


merged_df["final_label"] = merged_df.apply(get_final_label, axis=1)
# Sort as required: by cluster popularity, then alias popularity
cluster_sizes = merged_df.groupby("assigned_cluster").size().to_dict()
merged_df["cluster_popularity"] = merged_df["assigned_cluster"].map(cluster_sizes)
merged_df = merged_df.sort_values(
    ["cluster_popularity", "alias_nb_occ"], ascending=[False, False]
)

# 6. Save results

In [55]:
# Save to Excel
output_xlsx = (
    "data/W02/R04_evaluate_aliases_of_transformed_ema_registries/assessed_aliases_noise.xlsx"
)
output_dir = Path(output_xlsx).parent
output_dir.mkdir(parents=True, exist_ok=True)
merged_df.to_excel(output_xlsx, index=False)
print(f"Saved LLM-assessed alias pairs to {output_xlsx}")

Saved LLM-assessed alias pairs to data/W02/R04_evaluate_aliases_of_transformed_ema_registries/assessed_aliases_noise.xlsx


# 7. Results analysis

In [56]:
# reload data
output_xlsx = (
    "data/W02/R04_evaluate_aliases_of_transformed_ema_registries/assessed_aliases_noise.xlsx"
)
assessed_aliases_df = pd.read_excel(output_xlsx)

In [57]:
# Analysis: EMA registries with at least 1 or 2 'good' aliases
ema_good_alias_counts = (
    assessed_aliases_df[assessed_aliases_df["final_label"] == 1]
    .groupby("ema_full_name")
    .size()
)
for threshold in range(1, 6):
    count = (ema_good_alias_counts >= threshold).sum()
    alias_label = "alias" if threshold == 1 else "aliases"
    print(
        f"Number of transformed EMA registries with at least {threshold} good {alias_label}: {count}"
    )


Number of transformed EMA registries with at least 1 good alias: 4
Number of transformed EMA registries with at least 2 good aliases: 0
Number of transformed EMA registries with at least 3 good aliases: 0
Number of transformed EMA registries with at least 4 good aliases: 0
Number of transformed EMA registries with at least 5 good aliases: 0


In [58]:
# mean number of good aliases per EMA registry
mean_good_aliases = ema_good_alias_counts.mean()
print(f"Mean number of good aliases per transformed EMA registry: {mean_good_aliases:.2f}")
# mean number of wrong aliases per EMA registry
mean_wrong_aliases = (
    assessed_aliases_df[assessed_aliases_df["final_label"] == 0]
    .groupby("ema_full_name")
    .size()
    .mean()
)
print(f"Mean number of wrong aliases per transformed EMA registry: {mean_wrong_aliases:.2f}")

Mean number of good aliases per transformed EMA registry: 1.00
Mean number of wrong aliases per transformed EMA registry: 1.00


In [59]:
# mean % of good aliases (vs wrong aliases) per EMA registry
# for each EMA registry, calculate the ratio of good to total aliases
def calculate_good_alias_ratio(group):
    good_count = group[group["final_label"] == 1].shape[0]
    total_count = group.shape[0]
    return good_count / total_count if total_count > 0 else 0
ema_good_alias_ratios = (
    assessed_aliases_df.groupby("ema_full_name")
    .apply(calculate_good_alias_ratio)
)
mean_good_alias_ratio = ema_good_alias_ratios.mean()
print(f"Mean ratio of good aliases per transformed EMA registry: {mean_good_alias_ratio:.2%}")

Mean ratio of good aliases per transformed EMA registry: 5.71%


/tmp/ipykernel_29194/1533864510.py:9: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(calculate_good_alias_ratio)


In [60]:
# compute the same for but subsets of data
# first for EMA_registries that are in clusters with more than 50 aliases
n = 10
large_clusters_df = assessed_aliases_df[
    assessed_aliases_df["assigned_cluster"].isin(
        assessed_aliases_df["assigned_cluster"].value_counts()[lambda x: x > n].index
    )
]
# then for EMA_registries that are in clusters with less than 50 aliases
small_clusters_df = assessed_aliases_df[
    assessed_aliases_df["assigned_cluster"].isin(
        assessed_aliases_df["assigned_cluster"].value_counts()[lambda x: x <= n].index
    )
]

# compute Mean ratio of good aliases per transformed EMA registry for large clusters
large_clusters_good_alias_ratios = (
    large_clusters_df.groupby("ema_full_name")
    .apply(calculate_good_alias_ratio)
)
mean_large_clusters_good_alias_ratio = large_clusters_good_alias_ratios.mean()
print(f"Mean ratio of good aliases in large clusters: {mean_large_clusters_good_alias_ratio:.2%}")

# compute Mean ratio of good aliases per transformed EMA registry for small clusters
small_clusters_good_alias_ratios = (
    small_clusters_df.groupby("ema_full_name")
    .apply(calculate_good_alias_ratio)
)
mean_small_clusters_good_alias_ratio = small_clusters_good_alias_ratios.mean()
print(f"Mean ratio of good aliases in small clusters: {mean_small_clusters_good_alias_ratio:.2%}")


Mean ratio of good aliases in large clusters: 3.77%
Mean ratio of good aliases in small clusters: 11.76%


/tmp/ipykernel_29194/4016142208.py:19: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(calculate_good_alias_ratio)
/tmp/ipykernel_29194/4016142208.py:27: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(calculate_good_alias_ratio)
